https://adventofcode.com/2021/day/6

### Part 1:

In hindsight I was a bit lazy on this. I should have noticed that something including `exponential` means we are about to enter a territory that is computationally infeasible. 

Regardless my approach is pretty simple:
- Determine how many new fish to add (if any existing are at 0).
- Append new ones with life 9 (I do this since it needs to start at 8, and my `-1 day` happens at end of loop
- Anything that had been a 0 should be converted to a life 7 (same logic as above, this really becomes a 6)
- At end of loop subtract 1 life.

In [1]:
import numpy as np

# sample data
sample = [3,4,3,1,2]

arr = np.asarray(sample)

# iterate over 80 times
for i in range(80):

    # if anything is at 0 we add a new element starting at 8
    new_size = sum(arr == 0)
    
    if new_size > 0:
        arr = np.append(arr, new_size*[9])
    
    # convert any 0s to 6
    arr = np.where(arr == 0, 7, arr)
    
    # subtract 1 life at end of each
    arr = arr - 1
    
print(f"End of day {i}: {len(arr)}")
assert(len(arr) == 5934)

End of day 79: 5934


In [2]:
# read data 
with open('data/day06.txt') as fh:
    data = [line.split(',') for line in fh.readlines()]

# Ugly cleaning, but so it goes
start = [int(x) for x in data[0]]
arr = np.asarray(start)

# iterate over 80 days
for i in range(80):

    # if anything is at 0 we add a new element starting at 8
    new_size = sum(arr == 0)
    
    if new_size > 0:
        arr = np.append(arr, new_size*[9])
    
    # convert any 0s to 6
    arr = np.where(arr == 0, 7, arr)
    
    # subtract 1 life at end of each
    arr = arr - 1
    
print(f"End of day {i}: {len(arr)}")

End of day 79: 353079


### Part 2: Some Efficiency Needed

Now the problem is not feasible to solve by an ever-growing array. This is largely hinted at as the sample problem has shifted into a result of `26984457539`, so we know our solution will be significantly larger. 

#### Reframing Problem: 

It took me awhile to catch, but there is a nice pattern we can use here. I can think about the problem as `number of days until a new addition` rather than storing an ever-growing array.

Let's start with a simple example: `[3,4,3,1,2]`
- We know our current size is `5`
- We know that in 1 day we need to add 1 new fish, in 2 days we add 1 new fish, in 3 days we add 2 new fish, and in 4 days we add 1 fish
- We also know that whenever we add a new fish we need to:
    - Give it 8 days until IT adds a new fish
    - Give the original fish that is reproducing 6 more days until adding a new fish 

With this framework, we can store off a dictionary like below: 
```python
size_dict = {1: 1, 2: 1, 3: 2, 4: 1}
```

We can then step through each day, and when we hit a day with that matches our dictionary we take a few steps:
- Increment the total number of fish we have 
    - E.g. on Day 1 we go from 5 -> 6 in size
        - `Note Day 1 here really means Day 2 since I am working in start of 0`
- Add the new fish being added to `size_dict` twice:
    - Once in the `day+7` spot to represent the existing fish that will reproduce again in 6 days
    - Once in the `day+9` spot to represent the new fish that will reproduce in 8 days

#### Conclusion: 

The benefit here is that instead of relying on a massive array we can just store off a small dictionary as well as a single integer variable that incremements through each step. Very efficient

In [3]:
from collections import defaultdict
sum_d = defaultdict(lambda: 0)

sample = [3,4,3,1,2]

# initialize dict
for day in sample:
    sum_d[day] += 1
    

size = len(sample)
    
# for day of life 
for d in range(80):
    
    #print(f"On Day {d} size is {size}")
    
    # update based on sum_d dict
    adds = sum_d[d]
    sum_d[d+9] += adds # adding for future for new ones
    sum_d[d+7] += adds # adding for future for old ones
    size = size + adds
print(f"End of Day {d + 1} size is {size}")
assert(size == 5934)

End of Day 80 size is 5934


In [4]:
sum_d = defaultdict(lambda: 0)

sample = [3,4,3,1,2]

# initialize dict
for day in sample:
    sum_d[day] += 1
    

size = len(sample)
    
# for day of life 
for d in range(256):
    
    #print(f"On Day {d} size is {size}")
    
    # update based on sum_d dict
    adds = sum_d[d]
    sum_d[d+9] += adds # adding for future for new ones
    sum_d[d+7] += adds # adding for future for old ones
    size = size + adds
print(f"End of Day {d+1} size is {size}")
assert(size == 26984457539)

End of Day 256 size is 26984457539


In [5]:
sum_d = defaultdict(lambda: 0)

# read data 
with open('data/day06.txt') as fh:
    data = [line.split(',') for line in fh.readlines()]

start = [int(x) for x in data[0]]

# initialize dict
for day in start:
    sum_d[day] += 1
    
size = len(start)
    
# for day of life 
for d in range(256):
    # update based on sum_d dict
    adds = sum_d[d]
    sum_d[d+9] += adds # adding for future for new ones
    sum_d[d+7] += adds # adding for future for old ones
    size = size + adds
print(f"End of Day {d+1} size is {size}")

End of Day 256 size is 1605400130036
